# Neural Robot Dynamics Training on Colab

This notebook demonstrates how to setup the environment, generate a dataset, and train the NeRD model.

In [1]:
# 1. Setup Environment
!git clone https://github.com/dhruv0000/neural-robot-dynamics.git
%cd neural-robot-dynamics
!pip install -r requirements.txt
!pip install warp-lang==1.8.0

Cloning into 'neural-robot-dynamics'...
remote: Enumerating objects: 419, done.
remote: Counting objects: 100% (419/419), done.
remote: Compressing objects: 100% (307/307), done.
remote: Total 419 (delta 82), reused 405 (delta 68), pack-reused 0 (from 0)
Receiving objects: 100% (419/419), 17.27 MiB | 14.11 MiB/s, done.
Resolving deltas: 100% (82/82), done.
Filtering content: 100% (11/11), 202.03 MiB | 45.27 MiB/s, done.
/content/neural-robot-dynamics
ERROR: Ignored the following yanked versions: 1.11.0, 1.14.0rc1
ERROR: Ignored the following versions that require a different python version: 1.10.0 Requires-Python <3.12,>=3.8; 1.10.0rc1 Requires-Python <3.12,>=3.8; 1.10.0rc2 Requires-Python <3.12,>=3.8; 1.10.1 Requires-Python <3.12,>=3.8; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc

In [2]:
# 2. Generate Dataset
# We generate a smaller dataset for demonstration purposes.

%cd generate

# Generate Training Data
!python generate_dataset_contact_free.py --env-name Cartpole --num-transitions 10000 --dataset-dir ../data/datasets/ --dataset-name trajectory_len-100_train.hdf5 --trajectory-length 100 --num-envs 64 --seed 0

# Generate Validation Data
!python generate_dataset_contact_free.py --env-name Cartpole --num-transitions 2000 --dataset-dir ../data/datasets/ --dataset-name trajectory_len-100_valid.hdf5 --trajectory-length 100 --num-envs 64 --seed 10

%cd ..

/content/neural-robot-dynamics/generate
Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
 [NeuralEnvironment] Creating abstract contact environment: Cartpole. 
Creating 64 environments: 100% 64/64 [00:00<00:00, 104.68it/s]
Module warp.sim.integrator_featherstone 18b3327 load on device 'cuda:0' took 21015.98 ms  (compiled)
Module envs.abstract_contact_environment 8e8d790 load on device 'cuda:0' took 704.20 ms  (compiled)
Module integrators.integrator_neural ee402cd load on device 'cuda:0' took 742.43 ms  (compiled)
 [NeuralEnvironment] Created a DUMMY Neural Integrator. 
  0% 0/10000 [00:00<?, ?it/s]Module utils.warp_utils 294c46a load on device 'cuda:0' took 663.37 ms  (compile

In [3]:
# 3. Train Baseline Model (Transformer)
%cd train

import yaml
import os

# Load default config
with open('cfg/Cartpole/transformer.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

# Override dataset paths to point to the generated data
cfg['algorithm']['dataset']['train_dataset_path'] = '../data/datasets/Cartpole/trajectory_len-100_train.hdf5'
cfg['algorithm']['dataset']['valid_datasets']['exp_trajectory'] = '../data/datasets/Cartpole/trajectory_len-100_valid.hdf5'

# Reduce training parameters for quick demonstration
cfg['algorithm']['num_epochs'] = 5
cfg['algorithm']['num_iters_per_epoch'] = 100

# Save the modified config
with open('colab_config.yaml', 'w') as f:
    yaml.dump(cfg, f)

# Run training
!python train.py --cfg colab_config.yaml --logdir ../data/logs/baseline

/content/neural-robot-dynamics/train
Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
2025-11-23 16:08:13.618385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763914093.637752    1076 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763914093.643600    1076 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763914093.658678    1076

In [4]:
# 4. Train Mamba Model
# We use the same config but add the --novelty mamba flag
!python train.py --cfg colab_config.yaml --novelty mamba --logdir ../data/logs/mamba

Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
2025-11-23 16:08:28.238597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763914108.269830    1151 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763914108.279452    1151 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763914108.301454    1151 computation_placer.cc:177] computati

In [5]:
# 5. Train Unroll Model
# We use the same config but add the --novelty unroll flag
!python train.py --cfg colab_config.yaml --novelty unroll --logdir ../data/logs/unroll

Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
2025-11-23 16:08:42.175221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763914122.194564    1222 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763914122.200357    1222 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763914122.214802    1222 computation_placer.cc:177] computati

In [6]:
# 6. RL Evaluation
# Evaluate the trained models using the pretrained RL policy

# Baseline
print("Evaluating Baseline Model...")
!cd ../eval/eval_rl && python run_rl.py --rl-cfg ./cfg/Cartpole/cartpole.yaml --playback ../../pretrained_models/RL_policies/Cartpole/0/nn/CartpolePPO.pth --num-envs 1 --num-games 5 --env-mode neural --nerd-model-path ../../data/logs/baseline/nn/best_model.pt

# Mamba
print("Evaluating Mamba Model...")
!cd ../eval/eval_rl && python run_rl.py --rl-cfg ./cfg/Cartpole/cartpole.yaml --playback ../../pretrained_models/RL_policies/Cartpole/0/nn/CartpolePPO.pth --num-envs 1 --num-games 5 --env-mode neural --nerd-model-path ../../data/logs/mamba/nn/best_model.pt

# Unroll
print("Evaluating Unroll Model...")
!cd ../eval/eval_rl && python run_rl.py --rl-cfg ./cfg/Cartpole/cartpole.yaml --playback ../../pretrained_models/RL_policies/Cartpole/0/nn/CartpolePPO.pth --num-envs 1 --num-games 5 --env-mode neural --nerd-model-path ../../data/logs/unroll/nn/best_model.pt

Evaluating Baseline Model...
Traceback (most recent call last):
  File "/content/neural-robot-dynamics/eval/eval_rl/run_rl.py", line 30, in <module>
    from rl_games.torch_runner import Runner
ModuleNotFoundError: No module named 'rl_games'
Evaluating Mamba Model...
Traceback (most recent call last):
  File "/content/neural-robot-dynamics/eval/eval_rl/run_rl.py", line 30, in <module>
    from rl_games.torch_runner import Runner
ModuleNotFoundError: No module named 'rl_games'
Evaluating Unroll Model...
Traceback (most recent call last):
  File "/content/neural-robot-dynamics/eval/eval_rl/run_rl.py", line 30, in <module>
    from rl_games.torch_runner import Runner
ModuleNotFoundError: No module named 'rl_games'
